In [1]:
from sklearn.cluster import OPTICS
from sklearn import metrics
import numpy as np
from numpy import inf
import matplotlib.pyplot as plt
import seaborn as sns
import math
import hdbscan

def optics(hic,min_samples=5,metric='minkowski',cluster_method='xi',min_cluster_size=5,xi=0.05):
    db = OPTICS(
    min_samples=min_samples, 
    max_eps=inf, 
    metric=metric,
    cluster_method = cluster_method,
    min_cluster_size=min_cluster_size
    ).fit(hic)
    #db = hdbscan.HDBSCAN(metric='euclidean').fit(hic)

    label = db.labels_
    # clusters = db.cluster_hierarchy_
    print(label)
    n_clusters_ = len(set(label)) - (1 if -1 in label else 0)
    n_noise_ = list(label).count(-1)
    print("Estimated number of clusters: %d" % n_clusters_)
    print("Estimated number of noise points: %d" % n_noise_)
    x = []
    X = []
    for i in range(len(label)):
        if label[i] != -1:
            X.append(hic[i])
            x.append(label[i])
    #print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, x))
    return label

def boundaryPlot(labels):
    n = len(labels)
    boundary = np.zeros(n)
    i = 0
    label = 0
    start = 0
    while i < n:
        if labels[i] != -1:
            if labels[i] == label:
                boundary[i] = start
            else:
                start = i
                label = labels[i]
                boundary[i] = start
        else:
            boundary[i] = i
            label = -1
        i = i + 1
    return boundary

In [5]:
#IMR90 cell line
for ii in range(19,20):
    filename = "hic/IMR90/50kb/intra_KR/chr{}_50k_intra_KR_matrix.txt".format(ii)#n*n matrix
    hic=np.loadtxt(filename)
    hic = np.nan_to_num(hic)
    labels = optics(hic,4,'cosine','xi',15)
    tads = boundaryPlot(labels)
    outpath = "output/IMR90/caspian_subcompart/caspian_chr{}".format(ii)
    i = 0
    res = 50000
    with open(outpath, "w") as out:
        while i < len(tads):
            if tads[i] < i:
                start = i - 1
                while i < len(tads) and tads[i] == start:
                    end = i
                    i = i + 1
                if end-start>=14:
                    startbin = start * res
                    endbin = end * res
                    out.write("\t".join((str(start), str(end), str(end), str(endbin))) + "\n")
                # else:
                #     start=start-1
            i = i + 1
        out.close()

[-1 -1 -1 ... -1 -1 -1]
Estimated number of clusters: 25
Estimated number of noise points: 614


In [4]:
#GM12878 cell line
import numpy as np
hic=np.loadtxt("hic/GM12878/50kb/intra_KR/chr19_50k_intra_KR_matrix.txt") #n*n matrix
# for min_sample in np.arange(3,15,1):
labels = optics(hic,5,'manhattan','xi',16)
tads = boundaryPlot(labels)

outpath = "output/GM12878/caspian_subcompart/caspian_chr19.txt"
i = 0
res = 50000
with open(outpath, "w") as out:
    while i < len(tads):
        if tads[i] < i:
            start = i - 1
            while i < len(tads) and tads[i] == start:
                end = i
                i = i + 1
            if end-start>=10:
                startbin = start * res
                endbin = end * res
                out.write("\t".join((str(start), str(startbin), str(end), str(endbin))) + "\n")
            # else:
            #     start=start-1
        i = i + 1
    out.close()

# plt.figure(figsize=(5,5))
# sns.heatmap(data=hic[350:800,350:800], robust=True, cmap="Reds")
# plt.plot([round(tad-350) for tad in tads][350:800],linewidth=2.5)
# plt.xlabel("")
# plt.ylabel("")
# #plt.title('eps={}, min_samples={}'.format(eps, min_samples))

# plt.show()

[ 0 -1 -1 ... -1 -1 -1]
Estimated number of clusters: 26
Estimated number of noise points: 510
